# ETL

This notebook contains my solution for the ETL exercise of the Python Project for Data Engineering course from coursera. The file dealership_data contains CSV, JSON, and XML files for used car data which contain features named car_model, year_of_manufacture, price, and fuel.

## General

In [ ]:
#Import
import glob                    
import pandas as pd                 
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

#DEfine files names
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract

In [ ]:
#CSV extract function
def extract_from_csv(file):
    dataframe = pd.read_csv(file)
    return dataframe

#JSON extract function
def extract_from_json(file):
    dataframe = pd.read_json(file, lines=True)
    return dataframe 

# XML extract function 
def extract_from_xml(file):
    dataframe = pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price', 'fuel'])
    tree = ET.parse(file)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = float(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model": car_model, 
                                      "year_of_manufacture": year_of_manufacture,
                                      "price": price,
                                      "fuel": fuel}, ignore_index=True)
    return dataframe


def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform

In [ ]:
# Add the transform function below
def transform(data):
    data["price"] = round(data["price"], 2)
    return data

## Load

In [ ]:
def load(targetfile, data):
    data.to_csv(targetfile)

## Log

In [ ]:
# Add the log function below
def log(message):
    timestamp_format = '%H-%M-%S-%m:%d:%Y' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_transformed_data.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## ETL

In [ ]:
log("ETL Started")

log("Extract Started")
extracted_data = extract()
log("Extract Finished")

log("Transform Started")
transformed_data = transform(extracted_data)
log("Transform Fnished")

log("Load Started")
load(targetfile, transformed_data)
log("Load Finished")

log("ETL Finished")